## Deliverable 2. Create a Customer Travel Destinations Map.

In [132]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [133]:
import pandas as pd
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yeppoon,AU,-23.1333,150.7333,87.58,59,15,11.68,few clouds
1,1,Pavilosta,LV,56.8879,21.1859,37.31,74,91,15.41,overcast clouds
2,2,Rikitea,PF,-23.1203,-134.9692,77.63,69,100,10.80,overcast clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,38.86,93,75,14.97,moderate rain
4,4,Preobrazheniye,RU,42.9019,133.9064,27.73,72,100,4.21,overcast clouds


In [134]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Please enter the minimum desired temperature for your destination: "))
max_temp = float(input("Please enter the maximum desired temperature for your destination: "))


Please enter the minimum desired temperature for your destination: 75
Please enter the maximum desired temperature for your destination: 80


In [135]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
destination_cities_df = city_data_df.loc[
    (city_data_df['Max Temp'] >= min_temp) &
    (city_data_df['Max Temp'] <= max_temp)]
    


In [136]:
# 4a. Determine if there are any empty rows.
destination_cities_df.info()
len(destination_cities_df)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107 entries, 2 to 696
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           107 non-null    int64  
 1   City                 107 non-null    object 
 2   Country              107 non-null    object 
 3   Lat                  107 non-null    float64
 4   Lng                  107 non-null    float64
 5   Max Temp             107 non-null    float64
 6   Humidity             107 non-null    int64  
 7   Cloudiness           107 non-null    int64  
 8   Wind Speed           107 non-null    float64
 9   Current Description  107 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 9.2+ KB


107

In [137]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = destination_cities_df.dropna(how = "all")

In [138]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotels_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotels_df["Hotel Name"] = ""
hotels_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,77.63,overcast clouds,-23.1203,-134.9692,
15,Carutapera,BR,76.46,scattered clouds,-1.1950,-46.0200,
16,Cotonou,BJ,79.68,broken clouds,6.3654,2.4183,
17,Kapaa,US,79.48,scattered clouds,22.0752,-159.3190,
23,Arraial Do Cabo,BR,75.88,overcast clouds,-22.9661,-42.0278,
24,Hilo,US,75.40,overcast clouds,19.7297,-155.0900,
25,Mar Del Plata,AR,75.22,thunderstorm with light rain,-38.0023,-57.5575,
36,Souillac,MU,77.36,scattered clouds,-20.5167,57.5167,
41,Zuenoula,CI,76.14,overcast clouds,7.4292,-6.0431,
54,Oistins,BB,78.89,broken clouds,13.0667,-59.5333,


In [139]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame and save as location parameter
    lat = row["Lat"]
    long = row["Lng"]
    params["location"] = f"{lat}, {long}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotels_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError as e:
        print(f"No hotels founds in {row['City']}. Error: ", e)
        pass
        

No hotels founds in Takapau. Error:  list index out of range
No hotels founds in Barra Patuca. Error:  list index out of range
No hotels founds in Ler. Error:  list index out of range
No hotels founds in Uarini. Error:  list index out of range


In [140]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotels_df = hotels_df.loc[hotels_df['Hotel Name'] != ""]
len(clean_hotels_df)

103

In [141]:
clean_hotels_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,77.63,overcast clouds,-23.1203,-134.9692,People ThankYou
15,Carutapera,BR,76.46,scattered clouds,-1.1950,-46.0200,Lidera
16,Cotonou,BJ,79.68,broken clouds,6.3654,2.4183,Novotel Cotonou Orisha
17,Kapaa,US,79.48,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
23,Arraial Do Cabo,BR,75.88,overcast clouds,-22.9661,-42.0278,Pousada Porto Praia


In [142]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"


# 8b. Export the City_Data into a csv
clean_hotels_df.to_csv(output_data_file, index_label="City_ID")

In [143]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotels_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotels_df[["Lat", "Lng"]]

In [144]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))